**Architecture**

- Each position is converted to a binary bit-string of size 773. 
- There are two sides (White and Black), 6 piece types (pawn, knight, bishop, rook, queen, and king), and 64 squares. Therefore, in order to represent a position as a binary bit-string, we would require: 
    - 2 × 6 × 64 = 768 bits (this is known as bitboard representation).
    - Additional 5 bits that represent: 
        - the side to move (1 for White and 0 for Black)
        - castling rights 
            - White can castle kingside
            - White can castle queenside
            - Black can castle kingside
            - Black can castle queenside

$\implies$ Overall, 773 bits are required to represent a position.

In [46]:
import torch
import chess
import chess.svg
import chess.pgn
import numpy as np
from IPython.display import SVG

from models.MiniDeepchess import SiameseNetwork, AutoEncoder

# Testing Engine


In [47]:
!unzip ./checkpoints/checkpoins-2.zip 

unzip:  cannot find or open ./checkpoints/checkpoins-2.zip, ./checkpoints/checkpoins-2.zip.zip or ./checkpoints/checkpoins-2.zip.ZIP.


In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

engine = SiameseNetwork(
    feature_extractor=AutoEncoder,
    feature_extractor_layers=[773, 600, 400, 200, 100], 
    comparator_layers=[400, 200, 200, 100], 
    output_dim=2
)

engine.load_pretrained(
    feature_extractor_path='./checkpoints/checkpoints/best_autoencoder.pth', 
    comparator_path='./checkpoints/checkpoints/best_siamese.pth'
)

engine.eval()

Loaded comparator from ./checkpoints/checkpoints/best_siamese.pth
Loaded feature extractor from ./checkpoints/checkpoints/best_autoencoder.pth


SiameseNetwork(
  (feature_extractor): AutoEncoder(
    (encoder_layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=773, out_features=600, bias=True)
        (1): BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
      )
      (1): Sequential(
        (0): Linear(in_features=600, out_features=400, bias=True)
        (1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
      )
      (2): Sequential(
        (0): Linear(in_features=400, out_features=200, bias=True)
        (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01)
      )
      (3): Sequential(
        (0): Linear(in_features=200, out_features=100, bias=True)
        (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(neg

In [49]:
board = chess.Board()

def initialize_board(gamefile_path: str=None) -> chess.Board:
    if gamefile_path is None: 
        board = chess.Board()
    else:
        board = chess.pgn.read_game(open(gamefile_path))
        board = board.end().board()
    return board

In [50]:
def bitboard(board: chess.Board) -> np.ndarray: 
    """
    Converts a chess board to a bitboard representation.

    Args:
        board (chess.Board): A chess board object.
        
    Returns:
        np.ndarray: A bitboard representation of the chess board. (Size: 773)
        
    Examples:
        >>> board = chess.Board()
        >>> bitboard(board)
        array([ 0,  0,  0, ...,  0,  0,  0])    
    """
    
    bitboard = np.zeros(773)
    piece_idx = {
        'p': 0,
        'n': 1,
        'b': 2,
        'r': 3,
        'q': 4,
        'k': 5
    }
    
    for index in range(64): 
        if board.piece_at(index): 
            color = int(board.piece_at(index).color) + 1
            bitboard[(piece_idx[board.piece_at(index).symbol().lower()] + index * 6) * color] = 1

    bitboard[-1] = int(board.turn)
    bitboard[-2] = int(board.has_kingside_castling_rights(True))
    bitboard[-3] = int(board.has_kingside_castling_rights(False))
    bitboard[-4] = int(board.has_queenside_castling_rights(True))
    bitboard[-5] = int(board.has_queenside_castling_rights(False))

    return bitboard

In [51]:
def compare_boards(state_a, state_b):
    if not isinstance(state_a, chess.Board) or not isinstance(state_b, chess.Board):
        raise TypeError(f"compare_boards only accepts chess.Board. Got: {type(state_a)} vs {type(state_b)}")
    
    a_tensor = torch.tensor(bitboard(state_a)).to(device).view(1, -1).float()
    b_tensor = torch.tensor(bitboard(state_b)).to(device).view(1, -1).float()

    with torch.no_grad():
        comparison = engine(a_tensor, b_tensor)
        
        # First win, second loss
        if comparison[0][0] < comparison [0][1]: 
            return (state_a, state_b)
        else:
            return (state_b, state_a)

In [52]:
import chess
import random
import torch

# Assuming device and engine are defined globally as in your setup
# def compare_boards(state_a, state_b) is provided as above

def minimax(board, depth, is_max, alpha_board=None, beta_board=None):
    """
    Minimax algorithm with alpha-beta pruning using compare_boards.
    
    Args:
        board (chess.Board): Current board state.
        depth (int): Remaining search depth.
        is_max (bool): True if maximizing player, False if minimizing.
        alpha_board (chess.Board): Best board for maximizer (or None).
        beta_board (chess.Board): Best board for minimizer (or None).
    
    Returns:
        chess.Board: The best board state according to the player's perspective.
    """
    # Base case: leaf node or game over
    if depth == 0 or board.is_game_over():
        return board
    
    if is_max:
        best_board = None
        for move in board.legal_moves:
            board.push(move)
            child_board = minimax(board, depth - 1, False, alpha_board, beta_board)
            board.pop()
            
            # Initialize best_board if None
            if best_board is None:
                best_board = child_board
            # Update best_board if child_board is better
            elif compare_boards(child_board, best_board)[0] == child_board:
                best_board = child_board
            
            # Alpha update: if best_board is better than alpha_board, update alpha
            if alpha_board is None or compare_boards(best_board, alpha_board)[0] == best_board:
                alpha_board = best_board
            
            # Pruning: if beta_board exists and best_board is worse than beta_board, prune
            if beta_board is not None and compare_boards(beta_board, best_board)[0] == beta_board:
                return best_board  # Prune remaining branches
        
        return best_board
    
    else:  # Minimizing player
        worst_board = None
        for move in board.legal_moves:
            board.push(move)
            child_board = minimax(board, depth - 1, True, alpha_board, beta_board)
            board.pop()
            
            # Initialize worst_board if None
            if worst_board is None:
                worst_board = child_board
            # Update worst_board if child_board is worse
            elif compare_boards(worst_board, child_board)[0] == child_board:
                pass  # worst_board is worse, keep it
            else:
                worst_board = child_board
            
            # Beta update: if worst_board is worse than beta_board, update beta
            if beta_board is None or compare_boards(beta_board, worst_board)[0] == worst_board:
                beta_board = worst_board
            
            # Pruning: if alpha_board exists and worst_board is better than alpha_board, prune
            if alpha_board is not None and compare_boards(worst_board, alpha_board)[0] == alpha_board:
                return worst_board  # Prune remaining branches
        
        return worst_board

def get_engine_move(board, depth):
    """
    Get the best move for the engine using minimax.
    
    Args:
        board (chess.Board): Current board state.
        depth (int): Search depth.
    
    Returns:
        chess.Move: The best move for the engine.
    """
    best_move = None
    best_final_board = None
    
    for move in board.legal_moves:
        board.push(move)
        # After engine's move, it's the opponent's turn (min player)
        final_board = minimax(board, depth - 1, False)
        board.pop()
        
        if best_final_board is None or compare_boards(final_board, best_final_board)[0] == final_board:
            best_final_board = final_board
            best_move = move
    
    return best_move

In [ ]:
# Game testing setup
board = chess.Board()
move_cnt = 0
print("Starting game: Engine (White) vs Random (Black)")
while not board.is_game_over():
    if board.turn == chess.WHITE:
        # Engine's turn
        move = get_engine_move(board, depth=2)  # Depth 2 for reasonable computation
        print(f"Engine plays: {move}")
        board.push(move)
    else:
        # Random player's turn
        move = random.choice(list(board.legal_moves))
        print(f"Random plays: {move}")
        board.push(move)
    move_cnt += 1
    if move_cnt % 5 == 0:
        display(SVG(chess.svg.board(board=board, size=400)))
    # Optional: Print board state
    # print(board)
    # print()

# Game result
result = board.result()
print(f"Game over. Result: {result}")
print(board)